### "Layer" class. A Layer encapsulates a state (weights) and some computation (defined in the `call` method).

In [1]:
%config IPCompleter.greedy=True # for autotab in Jupyter

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

In [3]:
print(tf.__version__)

2.0.0


In [4]:
class Linear(Layer):
    """y = w*x + b"""
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        
        w_init = tf.random_normal_initializer() # random initialization
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype="float32"), trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,), dtype="float32"), trainable=True)
        
    # call method
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b # w*x + b

# Instantiate Layer
linear_layer = Linear(4,2)

# Layer can be treated as a function
# Input some data 
y = linear_layer(tf.ones((2,2)))
assert y.shape == (2,4)

# Weights are automatically tracked under the 'weights' property
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

In [19]:
print(y)

tf.Tensor(
[[ 0.06665818 -0.09086242 -0.01717797  0.02770316]
 [ 0.06665818 -0.09086242 -0.01717797  0.02770316]], shape=(2, 4), dtype=float32)


### It's good practice to create weights in a separate `build` method

In [17]:
class Linear(Layer):
    """y = w*x + b"""
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.units = units
    
    # add_weight method shortcut for creating weights
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), 
                                 initializer="random_normal",
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="random_normal",
                                 trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
# Instantiate layer
linear_layer = Linear(4) # this means 4 units (weights) for the layer

# This also calls "build(input_shape)" and creates the weights
y = linear_layer(tf.ones((2,2)))

In [14]:
print(tf.ones((2,2)))

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [18]:
print(y)

tf.Tensor(
[[ 0.06665818 -0.09086242 -0.01717797  0.02770316]
 [ 0.06665818 -0.09086242 -0.01717797  0.02770316]], shape=(2, 4), dtype=float32)


### Automatically retreive the gradients of the weights of a layer by calling it inside a GradientTape. With these gradients one can update the weights of a layer, manually or using an optimizer object


In [21]:
# Load MNIST from keras API
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices((x_train.reshape(60000,784).astype("float32")/255, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

In [27]:
# Instantiate linear Layer (above) with 10 units
linear_layer = Layer(10)

# Instantiate a logistic loss function that expects integer targets
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Instantiate an optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# Iterate over the batches of the dataset
for step, (x,y) in enumerate(dataset):
    
    # open a GradientTape (Tf 2 new feature)
    with tf.GradientTape() as tape:
        
        # do forward pass
        logits = linear_layer(x) # input data to layer this -> direction
        
        # compute loss for this batch
        loss = loss_fn(y, logits) # target vs computed
        
        # get gradients of weights with respect of loss
        gradients = tape.gradient(loss, linear_layer.trainable_weights)
        
    # Update weights of linear layer
    optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))
    
    # logging
    if step % 100 == 0:
        print(step, float(loss))

0 6.8388214111328125
100 6.844223976135254
200 6.849213600158691
300 6.866031169891357
400 6.840302467346191
500 6.853946685791016
600 6.849768161773682
700 6.848651885986328
800 6.848796367645264
900 6.839184761047363


### Weights created by layers can be either trainable or non-trainable. They're exposed in the layer properties `trainable_weights` and `non_trainable_weights. This layer has non-trainable weights

In [33]:
class ComputeSum(Layer):
    """Returns sum of the inputs"""
    
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        
        # Create non-trainable weight
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)
        
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0)) # add inputs
        return self.total
    
my_sum = ComputeSum(2)
x = tf.ones((2,2))

y = my_sum(x)
print(y.numpy()) # [2, 2]

y = my_sum(x)
print(y.numpy())

assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


###  Layers can be recursively nested to create bigger computation blocks. Each layer will track the weights of its sublayers (both trainable and non-trainable)

In [38]:
# This is a multilayer perceptron
# Reuse Linear class

class MLP(Layer):
    """Simple stack of linear layers"""
    
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)
        
    def call(self, inputs):
        # connect layers
        x = self.linear_1(inputs)
        x = tf.nn.relu(x) # activation
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)
    
mlp = MLP()

# first call will create the weights
y = mlp(tf.ones(shape=(3, 64)))

# weights are recursively tracked
assert len(mlp.weights) == 6